In [ ]:
"""
01_load_raw_data.py — Load raw Excel data into Delta tables

Requirements:
- openpyxl dependency installed on the cluster (Library → PyPI → openpyxl)
- SparkSession available as spark
"""

import os
import logging
import pandas as pd
from pyspark.sql import SparkSession

# ─── CONFIGURATION ──────────────────────────────────────────────────────────────

# Unity Catalog schema and tables
SCHEMA = "pbi_project.raw_data"

# Directory where your .xlsx files are located (works on your cluster)
BASE_PATH = "/Workspace/Repos/raw/project_dataviz_databricks_pbi/01_raw_data"

# Definition of files and, if necessary, columns to rename
FILES = {
    "orders":  {"filename": "orders.xlsx"},
    "items":   {"filename": "items.xlsx"},
    "targets": {
        "filename":    "targets.xlsx",
        "rename_cols": {"monthly revenue target": "monthly_revenue_target"}
    }
}

# ─── UTILITIES ──────────────────────────────────────────────────────────────────

def setup_logger() -> logging.Logger:
    """Set up a logger with INFO level and standard formatting."""
    logger = logging.getLogger("LoadRawData")
    if not logger.handlers:
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
        logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger

def read_excel_file(path: str, logger: logging.Logger) -> pd.DataFrame:
    """Read an .xlsx file via pandas, check existence, and log its shape."""
    if not os.path.exists(path):
        logger.error(f"File not found: {path}")
        raise FileNotFoundError(f"{path} does not exist")
    try:
        df = pd.read_excel(path, engine="openpyxl")
        logger.info(f"Loaded {os.path.basename(path)}: {df.shape[0]} rows × {df.shape[1]} cols")
        return df
    except Exception:
        logger.exception(f"Error reading Excel file: {path}")
        raise

# ─── MAIN FLOW ──────────────────────────────────────────────────────────────────

def main():
    logger = setup_logger()

    # 1) Spark Session (useful when running outside a notebook)
    spark = SparkSession.builder.appName("LoadRawData").getOrCreate()

    # 2) Ensure that the schema exists
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
    logger.info(f"Schema ready: {SCHEMA}")

    # 3) Ingest each file
    for table_name, cfg in FILES.items():
        file_path = os.path.join(BASE_PATH, cfg["filename"])
        logger.info(f"Processing '{table_name}' ← {file_path}")

        # 3.1) Read file
        df_pd = read_excel_file(file_path, logger)

        # 3.2) Convert to Spark DataFrame
        df_spark = spark.createDataFrame(df_pd)

        # 3.3) Rename columns if mapping is provided
        for old, new in cfg.get("rename_cols", {}).items():
            df_spark = df_spark.withColumnRenamed(old, new)
            logger.info(f"Renamed column '{old}' → '{new}'")

        # 3.4) Write to Delta
        full_name = f"{SCHEMA}.{table_name}"
        df_spark.write.mode("overwrite").format("delta").saveAsTable(full_name)
        logger.info(f"Written Delta table: {full_name}")

    logger.info("✔️ All raw data tables ingested successfully.")

if __name__ == "__main__":
    main()


NameError: name 'spark' is not defined